In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("../Python/Datasets/Iris.csv", index_col='Id').sample(frac=1).reset_index(drop=True)
data = pd.concat([data, pd.get_dummies(data['Species'])], axis=1).drop(columns=['Species'])
data.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Iris-setosa,Iris-versicolor,Iris-virginica
0,5.2,2.7,3.9,1.4,0,1,0
1,5.7,3.8,1.7,0.3,1,0,0
2,6.4,2.8,5.6,2.1,0,0,1
3,5.8,2.7,5.1,1.9,0,0,1
4,6.6,3.0,4.4,1.4,0,1,0


In [3]:
x = tf.placeholder(dtype=tf.float64, shape=(None, 4))
y = tf.placeholder(dtype=tf.float64, shape=(None, 3))

with tf.variable_scope("weights", reuse=tf.AUTO_REUSE):
    
    w1 = tf.get_variable("w_1", initializer=tf.random_normal(shape=(4,32), dtype=tf.float64))
    b1 = tf.get_variable("b_1", initializer=tf.random_normal(shape=(1,32), dtype=tf.float64))
    
    w2 = tf.get_variable("w_2", initializer=tf.random_normal(shape=(32,16), dtype=tf.float64))
    b2 = tf.get_variable("b_2", initializer=tf.random_normal(shape=(1,16), dtype=tf.float64))
    
    w3 = tf.get_variable("w_3", initializer=tf.random_normal(shape=(16,3), dtype=tf.float64))
    b3 = tf.get_variable("b_3", initializer=tf.random_normal(shape=(1,3), dtype=tf.float64))
    
    l1 = tf.nn.relu(tf.add(tf.matmul(x, w1), b1))
    l2 = tf.nn.relu(tf.add(tf.matmul(l1, w2), b2))
    out = tf.nn.relu(tf.add(tf.matmul(l2, w3), b3))
        
    loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=out, labels=y)
    optim = tf.train.AdamOptimizer().minimize(loss)
    
    acc = tf.reduce_mean(tf.cast((tf.equal(tf.argmax(y,1), tf.argmax(out,1))), dtype=tf.float64))
    
    tf.summary.scalar("loss", tf.reduce_mean(tf.nn.softmax(loss)))
    tf.summary.scalar("acc", acc)

In [4]:
split = int(0.7*data.shape[0])
# inputs = np.random.random(size=(10,4))
# outputs = np.eye(3)[np.random.choice(3, 10)]

input_train, output_train, input_test, output_test = data.values[:split,:-3], data.values[:split,-3:], data.values[split:,:-3], data.values[split:,-3:]

In [5]:
with tf.Session() as sess:
    
    writer = tf.summary.FileWriter("./logs/logs2/run3", sess.graph)
    merged = tf.summary.merge_all()
    
    sess.run(tf.global_variables_initializer())
    
    over_a, over_l = [],[]
    step_cnt = 0
    for ep in range(1,1000):
        ep_a, ep_l = [],[]
        for i in range(0,input_train.shape[0],5):
            summary, a, l, _ = sess.run([merged, acc, loss, optim], feed_dict={x:input_train[i:i+5], y:output_train[i:i+5]})
            writer.add_summary(summary, step_cnt)
            step_cnt+=1
            ep_a.append(a)
            ep_l.append(np.mean(l))
        print("Epoch :", ep)
        print("Accuracy : ", np.mean(ep_a), "Loss : ", np.mean(ep_l))
        print("--------------------------------------------------------------------------")
        over_l.extend(ep_l)
        over_a.extend(ep_a)
    
    s, a, l, _ = sess.run([merged, acc, loss, optim], feed_dict={x:input_test, y:output_test})
    print("Testing Accuracy : ", a)
    print("Testing loss : ", np.mean(l))

Epoch : 1
Accuracy :  0.419047619047619 Loss :  38.29680926789775
--------------------------------------------------------------------------
Epoch : 2
Accuracy :  0.4666666666666666 Loss :  34.43917862234916
--------------------------------------------------------------------------
Epoch : 3
Accuracy :  0.4666666666666667 Loss :  31.330707484278008
--------------------------------------------------------------------------
Epoch : 4
Accuracy :  0.36190476190476195 Loss :  26.689502601911304
--------------------------------------------------------------------------
Epoch : 5
Accuracy :  0.29523809523809524 Loss :  21.792553435059272
--------------------------------------------------------------------------
Epoch : 6
Accuracy :  0.2857142857142857 Loss :  17.958406394245067
--------------------------------------------------------------------------
Epoch : 7
Accuracy :  0.29523809523809524 Loss :  14.533431396329862
--------------------------------------------------------------------------

Epoch : 59
Accuracy :  0.9428571428571428 Loss :  0.1416216681727213
--------------------------------------------------------------------------
Epoch : 60
Accuracy :  0.9428571428571428 Loss :  0.13917033048077507
--------------------------------------------------------------------------
Epoch : 61
Accuracy :  0.9523809523809523 Loss :  0.13689618276658705
--------------------------------------------------------------------------
Epoch : 62
Accuracy :  0.9523809523809523 Loss :  0.13469322474129433
--------------------------------------------------------------------------
Epoch : 63
Accuracy :  0.9523809523809523 Loss :  0.13242389883801936
--------------------------------------------------------------------------
Epoch : 64
Accuracy :  0.9523809523809523 Loss :  0.13031882500874356
--------------------------------------------------------------------------
Epoch : 65
Accuracy :  0.9523809523809523 Loss :  0.1282483132030119
--------------------------------------------------------------

Epoch : 116
Accuracy :  0.9714285714285715 Loss :  0.07693918760261713
--------------------------------------------------------------------------
Epoch : 117
Accuracy :  0.9714285714285715 Loss :  0.07632320087659161
--------------------------------------------------------------------------
Epoch : 118
Accuracy :  0.9714285714285715 Loss :  0.0761188429510208
--------------------------------------------------------------------------
Epoch : 119
Accuracy :  0.9714285714285715 Loss :  0.07548389080378949
--------------------------------------------------------------------------
Epoch : 120
Accuracy :  0.9714285714285715 Loss :  0.07530895801662003
--------------------------------------------------------------------------
Epoch : 121
Accuracy :  0.9714285714285715 Loss :  0.07492580468646899
--------------------------------------------------------------------------
Epoch : 122
Accuracy :  0.9714285714285715 Loss :  0.07443265286649903
------------------------------------------------------

Epoch : 176
Accuracy :  0.9714285714285715 Loss :  0.06312380150930912
--------------------------------------------------------------------------
Epoch : 177
Accuracy :  0.9714285714285715 Loss :  0.06299825581473177
--------------------------------------------------------------------------
Epoch : 178
Accuracy :  0.9714285714285715 Loss :  0.06287472575321326
--------------------------------------------------------------------------
Epoch : 179
Accuracy :  0.9714285714285715 Loss :  0.06274262017183226
--------------------------------------------------------------------------
Epoch : 180
Accuracy :  0.9714285714285715 Loss :  0.0625993536586271
--------------------------------------------------------------------------
Epoch : 181
Accuracy :  0.9714285714285715 Loss :  0.062491942743665756
--------------------------------------------------------------------------
Epoch : 182
Accuracy :  0.9714285714285715 Loss :  0.06234363736885478
-----------------------------------------------------

Epoch : 235
Accuracy :  0.9714285714285715 Loss :  0.058082285072108275
--------------------------------------------------------------------------
Epoch : 236
Accuracy :  0.9714285714285715 Loss :  0.058018396887590834
--------------------------------------------------------------------------
Epoch : 237
Accuracy :  0.9714285714285715 Loss :  0.05795562839298627
--------------------------------------------------------------------------
Epoch : 238
Accuracy :  0.9714285714285715 Loss :  0.05788963294752593
--------------------------------------------------------------------------
Epoch : 239
Accuracy :  0.9714285714285715 Loss :  0.05782512103851366
--------------------------------------------------------------------------
Epoch : 240
Accuracy :  0.9714285714285715 Loss :  0.057760517152656836
--------------------------------------------------------------------------
Epoch : 241
Accuracy :  0.9714285714285715 Loss :  0.05769600293336499
--------------------------------------------------

...

Epoch : 762
Accuracy :  0.9714285714285715 Loss :  0.042738735045453086
--------------------------------------------------------------------------
Epoch : 763
Accuracy :  0.9714285714285715 Loss :  0.04272523349797764
--------------------------------------------------------------------------
Epoch : 764
Accuracy :  0.9714285714285715 Loss :  0.04271175133821635
--------------------------------------------------------------------------
Epoch : 765
Accuracy :  0.9714285714285715 Loss :  0.04269839239421046
--------------------------------------------------------------------------
Epoch : 766
Accuracy :  0.9714285714285715 Loss :  0.04268517612413931
--------------------------------------------------------------------------
Epoch : 767
Accuracy :  0.9714285714285715 Loss :  0.04267159523842464
--------------------------------------------------------------------------
Epoch : 768
Accuracy :  0.9714285714285715 Loss :  0.04265843961926878
----------------------------------------------------

Epoch : 819
Accuracy :  0.9714285714285715 Loss :  0.042002927572459305
--------------------------------------------------------------------------
Epoch : 820
Accuracy :  0.9714285714285715 Loss :  0.04199077154776034
--------------------------------------------------------------------------
Epoch : 821
Accuracy :  0.9714285714285715 Loss :  0.04197814827969789
--------------------------------------------------------------------------
Epoch : 822
Accuracy :  0.9714285714285715 Loss :  0.04196599357344033
--------------------------------------------------------------------------
Epoch : 823
Accuracy :  0.9714285714285715 Loss :  0.04195344899155111
--------------------------------------------------------------------------
Epoch : 824
Accuracy :  0.9714285714285715 Loss :  0.04194130200485339
--------------------------------------------------------------------------
Epoch : 825
Accuracy :  0.9714285714285715 Loss :  0.04192879634859957
----------------------------------------------------

Epoch : 879
Accuracy :  0.980952380952381 Loss :  0.04128595908787354
--------------------------------------------------------------------------
Epoch : 880
Accuracy :  0.980952380952381 Loss :  0.041274456646083896
--------------------------------------------------------------------------
Epoch : 881
Accuracy :  0.980952380952381 Loss :  0.04126297014622372
--------------------------------------------------------------------------
Epoch : 882
Accuracy :  0.980952380952381 Loss :  0.04125149263296383
--------------------------------------------------------------------------
Epoch : 883
Accuracy :  0.980952380952381 Loss :  0.041239966670808946
--------------------------------------------------------------------------
Epoch : 884
Accuracy :  0.980952380952381 Loss :  0.04122855862874063
--------------------------------------------------------------------------
Epoch : 885
Accuracy :  0.980952380952381 Loss :  0.04121709641375441
----------------------------------------------------------

Epoch : 938
Accuracy :  0.980952380952381 Loss :  0.04062813400515655
--------------------------------------------------------------------------
Epoch : 939
Accuracy :  0.980952380952381 Loss :  0.040617363861647285
--------------------------------------------------------------------------
Epoch : 940
Accuracy :  0.980952380952381 Loss :  0.04060658942863747
--------------------------------------------------------------------------
Epoch : 941
Accuracy :  0.980952380952381 Loss :  0.04059579091973507
--------------------------------------------------------------------------
Epoch : 942
Accuracy :  0.980952380952381 Loss :  0.040585038017284907
--------------------------------------------------------------------------
Epoch : 943
Accuracy :  0.980952380952381 Loss :  0.04057431459634204
--------------------------------------------------------------------------
Epoch : 944
Accuracy :  0.980952380952381 Loss :  0.04056353667670699
----------------------------------------------------------

Epoch : 996
Accuracy :  0.980952380952381 Loss :  0.04001980413785419
--------------------------------------------------------------------------
Epoch : 997
Accuracy :  0.980952380952381 Loss :  0.04000957909231116
--------------------------------------------------------------------------
Epoch : 998
Accuracy :  0.980952380952381 Loss :  0.03999941544773885
--------------------------------------------------------------------------
Epoch : 999
Accuracy :  0.980952380952381 Loss :  0.03998923605038464
--------------------------------------------------------------------------
Testing Accuracy :  0.9777777777777777
Testing loss :  0.11965941944341361
